# Лекция 3: Токенизация

## От текст към модели

**Продължителност:** 2-2.5 часа
**Предпоставка:** Лекция 2 (Езикови модели, word embeddings)
**Следваща лекция:** Невронни мрежи

---
## 1. Мотивация: Какво е "дума"?

### Проблемът с токенизацията

В Лекция 2 видяхме **Word2Vec** - мощен начин да представим думи като вектори. Но имаме проблем:

**Как точно дефинираме "дума"?**

### Предизвикателства

#### 1. Многодумови изрази
- "New York" - една или две думи?
- "ice cream" - едно понятие, две думи

#### 2. Контракции
- "don't" → "do" + "n't"?
- "I'm" → "I" + "am"?

#### 3. Редки думи
- "ChatGPT" не съществуваше преди 2022
- Какво правим с думи извън речника?

### Решения

```
Думи      → Прост, но огромен речник
Символи   → Малък речник, но губим семантика
Subwords  → Най-доброто от двата света! ✓
```

In [ ]:
# Импортиране на необходимите библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
import re
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Всички библиотеки са заредени успешно!")

---
## 2. Прости подходи за токенизация

### 2.1 Word-level токенизация

**Предимства:**
- ✅ Прост и интуитивен
- ✅ Семантично значими единици

**Недостатъци:**
- ❌ Огромен речник (100,000+ думи)
- ❌ Out-of-vocabulary (OOV) проблем
- ❌ Няма споделяне на параметри

### 2.2 Character-level токенизация

**Предимства:**
- ✅ Малък речник (26 букви + пунктуация)
- ✅ Няма OOV проблем

**Недостатъци:**
- ❌ Много дълги последователности
- ❌ Символите нямат семантика
- ❌ По-трудно за модела да научи

In [ ]:
# Сравнение на word-level и character-level токенизация

example_text = "Hello, world! The weather is wonderful."

print("📝 Примерен текст:")
print(f"'{example_text}'\n")
print("="*70)

# Word-level
word_tokens = example_text.split()
print("\n1️⃣ Word-level токенизация:")
print(f"Токени: {word_tokens}")
print(f"Брой токени: {len(word_tokens)}")

# Character-level
char_tokens = list(example_text)
print("\n2️⃣ Character-level токенизация:")
print(f"Първите 20 токена: {char_tokens[:20]}")
print(f"Брой токени: {len(char_tokens)}")

print("\n" + "="*70)
print("\n📊 Сравнение:")
comparison = pd.DataFrame({
    'Метод': ['Word-level', 'Character-level'],
    'Брой токени': [len(word_tokens), len(char_tokens)],
    'Размер на речника': [len(set(word_tokens)), len(set(char_tokens))]
})
print(comparison.to_string(index=False))

---
## 3. Subword Tokenization: Byte-Pair Encoding (BPE)

### Основна идея

**BPE** е алгоритъм за компресия, адаптиран за NLP.

**Философия:**
- Започваме с символи
- Итеративно **сливаме най-честата двойка**
- Повтаряме докато достигнем желания размер

### BPE Алгоритъм

**Обучение:**
```
1. Започни с символи
2. Докато речник < размер:
   a. Намери най-честата двойка
   b. Слей ги в нов символ
   c. Замени всички срещания
```

**Използва се в:**
- GPT-2, GPT-3, GPT-4
- LLaMA
- RoBERTa

In [ ]:
# Проста имплементация на BPE

class SimpleBPE:
    def __init__(self):
        self.vocab = set()
        self.merges = {}
        self.merge_order = []

    def get_pairs(self, word):
        pairs = set()
        for i in range(len(word) - 1):
            pairs.add((word[i], word[i+1]))
        return pairs

    def train(self, corpus, num_merges):
        # Започваме с думи разделени на символи
        vocab = {}
        for word, freq in corpus.items():
            word_chars = list(word) + ['</w>']
            vocab[tuple(word_chars)] = freq
            self.vocab.update(word_chars)

        print(f"Начален речник: {sorted(self.vocab)}")
        print(f"Размер: {len(self.vocab)}\n")

        # Merge операции
        for merge_num in range(num_merges):
            pair_counts = Counter()
            for word, freq in vocab.items():
                pairs = self.get_pairs(word)
                for pair in pairs:
                    pair_counts[pair] += freq

            if not pair_counts:
                break

            best_pair = max(pair_counts, key=pair_counts.get)
            merged = ''.join(best_pair)

            self.merges[best_pair] = merged
            self.merge_order.append((best_pair, merged))
            self.vocab.add(merged)

            print(f"Итерация {merge_num + 1}:")
            print(f"  Merge: '{best_pair[0]}' + '{best_pair[1]}' → '{merged}'")
            print(f"  Честота: {pair_counts[best_pair]}")

            # Обновяваме речника
            new_vocab = {}
            for word, freq in vocab.items():
                new_word = []
                i = 0
                while i < len(word):
                    if i < len(word) - 1 and (word[i], word[i+1]) == best_pair:
                        new_word.append(merged)
                        i += 2
                    else:
                        new_word.append(word[i])
                        i += 1
                new_vocab[tuple(new_word)] = freq
            vocab = new_vocab

        print(f"\n✓ Финален речник: {len(self.vocab)} токена")

    def tokenize(self, word):
        word = list(word) + ['</w>']
        for pair, merged in self.merge_order:
            i = 0
            new_word = []
            while i < len(word):
                if i < len(word) - 1 and word[i] == pair[0] and word[i+1] == pair[1]:
                    new_word.append(merged)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            word = new_word
        return word

print("✓ SimpleBPE клас дефиниран!")

In [ ]:
# Обучаваме BPE

corpus = {
    'low': 5,
    'lower': 2,
    'newest': 6,
    'widest': 3,
}

print("📚 Корпус:")
for word, freq in corpus.items():
    print(f"  '{word}': {freq}")
print()

bpe = SimpleBPE()
bpe.train(corpus, num_merges=8)

In [ ]:
# Тестваме токенизацията

print("\n🧪 Тестване\n")

test_words = ['lowest', 'newer', 'wider', 'low']

for word in test_words:
    tokens = bpe.tokenize(word)
    print(f"'{word}': {tokens}")

---
## 4. Токенизация в съвременните LLM

### Популярни модели

| Модел | Алгоритъм | Vocab Size |
|-------|-----------|------------|
| GPT-2/3 | BPE | 50,257 |
| BERT | WordPiece | 30,522 |
| LLaMA | BPE | 32,000 |
| T5 | Unigram | 32,000 |

### Byte-level BPE

**Модерният подход:**
- Работи на byte ниво (UTF-8)
- 256 базови tokens (bytes)
- **Никога няма unknown token!**

### Context Window

**Важно:**
- "8k context" = 8,000 ТОКЕНА, не думи!
- API costs базирани на tokens
- Средно 1 token ≈ 0.75 думи (английски)

---
## 3.5 Byte-Level BPE: Модерният подход

### Проблемът с Character-Level BPE

**Character-level BPE има ограничения:**
- Unicode има 140,000+ символа
- Как обработваме emoji? 🤖🚀🎉
- Как обработваме специални символи? ™️, ©, ®
- Как обработваме текст на различни езици?

**Пример:**
```
"Hello 世界 مرحبا 🌍" 
```
- Латиница, китайски, арабски, emoji
- Character-level BPE трябва да знае всички тези символи!

### Решението: Byte-Level Encoding

**Ключова идея:**
- Всеки текст се кодира като **bytes** (UTF-8)
- UTF-8 използва 1-4 bytes за всеки Unicode символ
- Възможни byte стойности: 0-255 (само 256!)
- BPE работи на byte ниво вместо character ниво

**Предимства:**
- ✅ **Универсално:** Може да представи ВСЕКИ текст
- ✅ **Малък базов речник:** Само 256 bytes
- ✅ **Никога няма [UNK]:** Всеки символ може да бъде представен
- ✅ **Използва се от GPT-2/3/4**

### UTF-8 Encoding Basics

**UTF-8 правила:**
- ASCII символи (a-z, 0-9): 1 byte
- Европейски символи (á, é, ñ): 2 bytes
- Кирилица, китайски, арабски: 2-3 bytes
- Emoji: 3-4 bytes

**Пример:**
```
"Hello" → [72, 101, 108, 108, 111] (5 bytes)
"Здравей" → 14 bytes (кирилица)
"你好" → 6 bytes (китайски)
"🤖" → 4 bytes (emoji)
```

### Нека да видим това в действие!

In [ ]:
# Демонстрация на UTF-8 encoding

def show_utf8_encoding(text):
    """Показва как текст се кодира като bytes"""
    print(f"Текст: '{text}'")
    print(f"Дължина (символи): {len(text)}")
    
    # Кодиране в UTF-8 bytes
    encoded = text.encode('utf-8')
    print(f"UTF-8 bytes: {list(encoded)}")
    print(f"Дължина (bytes): {len(encoded)}")
    print(f"Hex представяне: {encoded.hex()}")
    
    # Показваме всеки символ
    print("\nСимвол по символ:")
    for char in text:
        char_bytes = char.encode('utf-8')
        print(f"  '{char}' → {list(char_bytes)} ({len(char_bytes)} bytes)")
    print()

print("🔤 UTF-8 Encoding Демонстрация\n")
print("="*70)

# ASCII текст
print("\n1️⃣ ASCII текст (1 byte на символ):")
show_utf8_encoding("Hello")

# Кирилица
print("="*70)
print("\n2️⃣ Кирилица (2 bytes на символ):")
show_utf8_encoding("Здравей")

# Смесен текст
print("="*70)
print("\n3️⃣ Смесен текст:")
show_utf8_encoding("Hello Здравей")

# Emoji
print("="*70)
print("\n4️⃣ Emoji (3-4 bytes):")
show_utf8_encoding("🤖🚀")

# Китайски
print("="*70)
print("\n5️⃣ Китайски (3 bytes на символ):")
show_utf8_encoding("你好世界")

In [ ]:
# Byte-Level BPE: Как работи

print("🔧 Byte-Level BPE Process\n")
print("="*70)

text = "Hello 🤖"

print(f"Оригинален текст: '{text}'\n")

# Стъпка 1: UTF-8 encoding
utf8_bytes = text.encode('utf-8')
print(f"Стъпка 1 - UTF-8 encoding:")
print(f"  Bytes: {list(utf8_bytes)}")
print(f"  Брой bytes: {len(utf8_bytes)}\n")

# Стъпка 2: Байтовете са началният "речник"
print(f"Стъпка 2 - Базов речник:")
print(f"  Възможни bytes: 0-255 (256 токена)")
print(f"  Текущи bytes: {set(utf8_bytes)}\n")

# Стъпка 3: BPE merge операции
print(f"Стъпка 3 - BPE merges:")
print(f"  BPE сега работи на byte последователности")
print(f"  Често срещани byte последователности се сливат")
print(f"  Например: [72, 101] (He) може да се слее в един токен\n")

# Стъпка 4: Mapping обратно
print(f"Стъпка 4 - Визуализация:")
print(f"  GPT-2 използва специално mapping:")
print(f"  Bytes се mapват към printable Unicode символи")

# GPT-2 byte encoder пример
gpt2_byte_to_unicode_sample = {
    33: '!', 34: '"', 35: '#', 72: 'H', 101: 'e', 108: 'l', 111: 'o'
}
print(f"  Пример mapping: {gpt2_byte_to_unicode_sample}")

print("\n💡 Резултат:")
print("   • Никога няма неизвестни символи")
print("   • Работи с ВСЕКИ Unicode текст")
print("   • GPT-2/3/4 използват този подход")

### Byte-Level vs Character-Level BPE

**Сравнение:**

| Характеристика | Character-Level | Byte-Level |
|---------------|-----------------|------------|
| Базов речник | ~100 chars | 256 bytes |
| Unicode поддръжка | Частична | Пълна |
| Unknown tokens | Възможни | Никога |
| Emoji | Проблематични | Работят перфектно |
| Multilingual | Изисква голям vocab | Работи навсякъде |
| Използва се в | Стари модели | GPT-2/3/4, LLaMA |

### GPT-2 Byte-Level BPE Детайли

**GPT-2 речник:**
```
50,257 tokens = 256 bytes + 50,000 merges + 1 special token
```

**Защо 256 bytes?**
- UTF-8 използва bytes (0-255)
- Всеки възможен byte е отделен token
- Гарантира че ВСЕКИ текст може да бъде кодиран

**Byte-to-Unicode Mapping:**
- GPT-2 използва clever trick
- Mapва bytes към printable Unicode range
- Позволява визуализация на tokens
- Избягва проблеми с контролни символи

### Практически пример

**Въпрос:** Как GPT-2 токенизира "Hello 🤖"?

**Отговор:**
1. "Hello " → Често срещано → 1 token
2. "🤖" → 4 bytes → вероятно 1-2 tokens (зависи от merges)

**За разлика от character-level:**
- Character-level може да не знае какво е 🤖
- Byte-level ВИНАГИ може да го кодира (4 bytes)

### Защо това е революционно

**Преди Byte-Level BPE:**
- Модели трябваше да "знаят" всички възможни символи
- Нови emoji = проблем
- Специални символи = [UNK]
- Различни езици = огромен vocab

**След Byte-Level BPE:**
- ✅ Работи с ВСЕКИ текст
- ✅ Никога няма [UNK]
- ✅ Нови символи = няма проблем
- ✅ Универсално решение

**Това е една от причините GPT-2/3/4 да са толкова мощни!**

In [ ]:
# Сравнение: Character vs Byte representation

print("📊 Character-Level vs Byte-Level Comparison\n")
print("="*70)

test_strings = [
    "Hello",
    "Здравей", 
    "你好",
    "مرحبا",
    "🤖🚀"
]

results = []

for text in test_strings:
    char_count = len(text)
    byte_count = len(text.encode('utf-8'))
    bytes_per_char = byte_count / char_count
    
    results.append({
        'Текст': text,
        'Символи': char_count,
        'Bytes': byte_count,
        'Bytes/Char': f'{bytes_per_char:.1f}'
    })

df = pd.DataFrame(results)
print(df.to_string(index=False))

print("\n" + "="*70)
print("\n💡 Наблюдения:")
print("   • ASCII: 1 byte на символ")
print("   • Кирилица: 2 bytes на символ")
print("   • Китайски: 3 bytes на символ")
print("   • Emoji: 4 bytes на символ")
print("\n   → Character-level vocab трябва да съдържа всички символи")
print("   → Byte-level vocab винаги е само 256!")

# Визуализация
plt.figure(figsize=(10, 6))
languages = [r['Текст'] for r in results]
char_counts = [r['Символи'] for r in results]
byte_counts = [r['Bytes'] for r in results]

x = np.arange(len(languages))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width/2, char_counts, width, label='Символи', color='steelblue', alpha=0.8)
bars2 = ax.bar(x + width/2, byte_counts, width, label='Bytes', color='coral', alpha=0.8)

ax.set_xlabel('Текст', fontsize=12)
ax.set_ylabel('Брой', fontsize=12)
ax.set_title('Character Count vs Byte Count', fontsize=14, pad=15)
ax.set_xticks(x)
ax.set_xticklabels(languages)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n✓ Byte-level BPE решава проблема с multilingual text!")

In [ ]:
# Демонстрация с модерни токенизатори

try:
    from transformers import AutoTokenizer

    print("🔬 Сравнение на токенизатори\n")

    text = "Hello! GPT-3 and BERT use different tokenizers."
    print(f"Текст: '{text}'\n")

    # GPT-2
    try:
        gpt2 = AutoTokenizer.from_pretrained('gpt2')
        gpt2_tokens = gpt2.tokenize(text)
        print(f"GPT-2 ({len(gpt2_tokens)} tokens):")
        print(f"  {gpt2_tokens}")
    except:
        print("GPT-2: не може да се зареди")

    # BERT
    try:
        bert = AutoTokenizer.from_pretrained('bert-base-uncased')
        bert_tokens = bert.tokenize(text)
        print(f"\nBERT ({len(bert_tokens)} tokens):")
        print(f"  {bert_tokens}")
    except:
        print("\nBERT: не може да се зареди")

except ImportError:
    print("⚠️  transformers не е инсталиран")
    print("   pip install transformers")

---
## 5. Обобщение

### 🎓 Ключови изводи

**1. Токенизацията е критична**
- Балансираме размер на речника vs дължина на seq
- Word-level: прост, но OOV проблем
- Character-level: няма OOV, но дълги sequences

**2. Subword tokenization е стандартът**
- **BPE:** Най-популярен (GPT, LLaMA)
  - Greedy, frequency-based
  - Детерминистичен
- **Unigram:** Вероятностен (T5)
- **WordPiece:** BERT

**3. Byte-level BPE**
- Работи на byte ниво
- Никога няма UNK tokens
- Използва се от GPT-2/3/4

**4. Практически**
- Context windows са в ТОКЕНИ
- API costs зависят от tokens
- Езикова пристрастност е проблем

### 🔜 Следваща лекция: Невронни мрежи

**Вече знаем как да токенизираме. Какво следва?**

- Как обработваме последователности от токени?
- Невронни мрежи и backpropagation
- Подготовка за Transformers!

### 📚 Препоръчително четене

1. **"Neural Machine Translation of Rare Words with Subword Units"** - Sennrich et al. (2016)
2. **"Language Models are Unsupervised Multitask Learners"** - Radford et al. (2019) - GPT-2
3. **HuggingFace Tokenizers Documentation**
4. **OpenAI Tokenizer Tool** - https://platform.openai.com/tokenizer

---
## Край на Лекция 3

### Благодаря за вниманието! 🎉

**Въпроси? 🤔**